# Using AstraPy with vector database

Work with AstraPy

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

### Set up your python environment




In [ ]:
%pip install astrapy --pre --upgrade


### Import needed libraries



In [2]:
from astrapy.base import AstraClient, http_methods
from astrapy.collections import AstraCollectionClient
from astrapy.ops import AstraOps
import os
import uuid




### Astra Setup



### Astra Variables - REPLACE THE TOKENS IN THE CODE WITH YOUR INFORMATION

In [ ]:
# Input your database keyspace name:
ASTRA_DB_ID = <astra_db_id>
ASTRA_DB_APPLICATION_TOKEN = <AstraCS:....>
ASTRA_DB_REGION = <astra_db_region>


# No need to change these values
ASTRA_DB_KEYSPACE = "default_keyspace"
TEST_COLLECTION_NAME = "collection_test"

In [ ]:
### Client setup
astra_client = AstraClient(
        astra_database_id=ASTRA_DB_ID,
        astra_database_region=ASTRA_DB_REGION,
        astra_application_token=ASTRA_DB_APPLICATION_TOKEN,
    )

collections_client = AstraCollectionClient(astra_client=astra_client)
test_namespace = collections_client.namespace(ASTRA_DB_KEYSPACE)
test_collection = collections_client.namespace(ASTRA_DB_KEYSPACE).collection(TEST_COLLECTION_NAME)

In [ ]:
# Create and view collections
# Just in case it already exists, delete it
res = test_namespace.delete_collection(name=TEST_COLLECTION_NAME)
print (res)

res = test_namespace.create_collection(name=TEST_COLLECTION_NAME, size=5)
print(res)

res = test_namespace.get_collections()
print(res)

# Insert one document

In [ ]:
# Insert into vector collection
json_query = {
        "_id": "4",
        "name": "Coded Cleats Copy",
        "description": "ChatGPT integrated sneakers that talk to you",
        "$vector": [0.25, 0.25, 0.25, 0.25, 0.25],
    }

res = test_collection.insert_one(document=json_query)
print(res)

In [ ]:
# Insert non-vector document

json_query = {
        "_id": "Cliff1",
        "first_name": "Cliff",
        "last_name": "Wicklow",
    }
response = test_collection.insert_one(document=json_query)
print(response)


# Insert multiple documents

In [ ]:
json_query = [
        {
            "_id": "1",
            "name": "Coded Cleats",
            "description": "ChatGPT integrated sneakers that talk to you",
            "$vector": [0.1, 0.15, 0.3, 0.12, 0.05],
        },
        {
            "_id": "2",
            "name": "Logic Layers",
            "description": "An AI quilt to help you sleep forever",
            "$vector": [0.45, 0.09, 0.01, 0.2, 0.11],
        },
        {
            "_id": "3",
            "name": "Vision Vector Frame",
            "description": "Vision Vector Frame - A deep learning display that controls your mood",
            "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
        },
    ]

res = test_collection.insert_many(documents=json_query)
print(res)

# Find documents

## Find a document

In [ ]:
document = test_collection.find_one(filter={"_id":"4"})
print(document)

## Find documents using vector search

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
options = {"limit": 100}

document = test_collection.find(sort=sort, options=options)
print(document)

## Find documents using vector search and projection

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
options = {"limit": 100}
projection = {"$vector": 1, "$similarity": 1}

document = test_collection.find(sort=sort, options=options, projection=projection)
print(document)




## Find one document using vector search and projection

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
projection = {"$vector": 1}

document = test_collection.find_one(sort=sort, options={}, projection=projection)
print(document)
    

## Find one and update with vector search

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
update = {"$set": {"status": "active"}}
options = {"returnDocument": "after"}

test_collection.find_one_and_update(sort=sort, update=update, options=options)
document = test_collection.find_one(filter={"status": "active"})
print(document)
 

## Find one and replace with vector search

In [ ]:
sort = ({"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]},)
replacement = {
        "_id": "3",
        "name": "Vision Vector Frame",
        "description": "Vision Vector Frame - A deep learning display that controls your mood",
        "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
        "status": "inactive",
}
options = {"returnDocument": "after"}

test_collection.find_one_and_replace(sort=sort, replacement=replacement, options=options)
document = test_collection.find_one(filter={"name": "Vision Vector Frame"})
print(document)
    

## Delete a collection

In [ ]:
returnval = test_namespace.delete_collection(name=TEST_COLLECTION_NAME)
print(returnval)